In [186]:
from IPython.display import clear_output, display
from ipywidgets import ToggleButton, Button, HBox, VBox, Output, interactive
import bibtexparser
from bibtexparser.bwriter import BibTexWriter
from bibtexparser.bibdatabase import BibDatabase
import webbrowser
from urllib.parse import urlencode
import os

## Criteria
In order to be relevant for the literature analysis a publication must fullfill atleast one of the inclusion criteria and none of the exclusion criteria.

In [162]:
inclusion_criteria = ['Reviews the current state of explainability research', 'Presents a specific method for enhancing explainability for models']
exclusion_criteria = ['Is not scientific literature', 'Does not describe the used explainability method', 'The publication does not focus on explainability', 'The described method is neither general, nor focused on NLP']

In [195]:
def printFirstProject(entries):
    if (len(entries) > 0):
        print(entries[0]['title'])
        print()
        print(entries[0]['abstract'] if 'abstract' in entries[0] else webbrowser.open('https://doi.org/' + entries[0]['doi']))
    else:
        print('Finished!')

def dumpBibfile(name, entries):
    db = BibDatabase()
    db.entries = entries

    writer = BibTexWriter()
    with open(f'../data/{name}.bib', 'w') as bibfile:
        bibfile.write(writer.write(db))

def openIfExistsOtherwiseNew(name):
    if( os.path.isfile(f'../data/{name}.bib')):
        with open(f'../data/{name}.bib') as file:
            return bibtexparser.bparser.BibTexParser(common_strings=True).parse_file(file).entries
    else:
        return []

In [196]:
stage1_entries = openIfExistsOtherwiseNew('stage2')

stage1_rejected_entries = openIfExistsOtherwiseNew('stage1_rejected')

stage1_interesting_entries = openIfExistsOtherwiseNew('stage1_interesting')
stage2_entries = []
    

    

In [211]:
layout = widgets.Layout(width='auto', height='40px')

outputfield = Output()
with outputfield:
    printFirstProject(stage1_entries)
inclusion = [ToggleButton(description=criteria, button_style='success', layout=layout) for criteria in inclusion_criteria]
exclusion = [ToggleButton(description=criteria, button_style='danger', layout=layout) for criteria in exclusion_criteria]
interesting = ToggleButton(description='Not relevant but interesting', button_style='info', layout=layout)
submit = Button(description='Submit')
ui = VBox([outputfield, HBox(inclusion), HBox(exclusion), interesting, submit])

def submitted(b):
    # accept or reject publication
    entry = stage1_entries.pop(0)
    entry['comments'] = ', '.join([crit.description for crit in inclusion if crit.value]) + ', ' + ', '.join([crit.description for crit in exclusion if crit.value])
    if (any([crit.value for crit in inclusion]) and all([not crit.value for crit in exclusion])):
        stage2_entries.append(entry)
    elif (interesting.value):
        stage1_interesting_entries.append(entry)
    else:
        stage1_rejected_entries.append(entry)
    
    # show next project and reset toggle states
    with outputfield:
        clear_output()
        printFirstProject(stage1_entries)
    for crit in inclusion:
        crit.value=False
    for crit in exclusion:
        crit.value=False
    interesting.value=False

submit.on_click(submitted)

display(ui)

In [222]:
dumpBibfile('stage1_rejected', stage1_rejected_entries)
dumpBibfile('stage2', stage2_entries)
dumpBibfile('stage1_interesting', stage1_interesting_entries)

In [223]:
len(stage2_entries) 

90